In [ ]:
# We are going to use an RGB lightcell to control the wheels here

In [53]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [30]:
%sendtofile --source /home/julian/repositories/jupyter_micropython_developer_notebooks/Sensor_Kennel/TSL561_funcs.py

Sent 54 lines (1757 bytes) to TSL561_funcs.py.


In [29]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      test.mosquitto.org
pinled          2
boardname       miniwheels2

Sent 5 lines (137 bytes) to config.txt.


(104, 13)


In [54]:
%sendtofile main.py

from machine import Pin, I2C
from TSL561_funcs import setupTSL561, setregtimings, luminosity
from mqtt_as import MQTTClient, config
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
i2c = I2C(scl=Pin(5), sda=Pin(4))  # P5=D1=SCL, P4=D2=SDA
setupTSL561(i2c)
setregtimings(True, 0)  #integration_time = 0  # 0:13.7ms, 1:101ms, 2:402ms, or 3:manual

boardname = fconfig["boardname"]
tlux = boardname.encode() + b"/lux"
luxmsg = None

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 1600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)

async def luxtask():
    global luxmsg
    br, ir = luminosity()
    while True:
        try:
            nbr, nir = luminosity()
        except OSError as e:
            print(e)
            await asyncio.sleep_ms(1000)
            continue
        if abs(nbr - br) >= 2 or abs(nir - ir) > 2:
            br, ir = nbr, nir
            print((br, ir))
            luxmsg = "%d %d" % (br, ir)
        await asyncio.sleep_ms(5)
        
async def publishtask():
    global luxmsg
    while True:
        lluxmsg, luxmsg = luxmsg, None
        if lluxmsg is not None:
            await client.publish(tlux, lluxmsg)
        await asyncio.sleep_ms(2)

async def immediateluxpublishtask():
    while True:
        try:
            br, ir = luminosity()
        except OSError as e:
            print(e)
            await asyncio.sleep_ms(200)
            continue
        luxmsg = "%d %d" % (br, ir)
        await client.publish(tlux, luxmsg)
        
async def mqtttask(bflip=False):
    print("mqttttask", bflip)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        await client.connect()
        return
    except OSError as e:
        print("client connect fail", [e])
    aloop.create_task(mqtttask(True))
        
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(ledtask())
#aloop.create_task(luxtask())
#aloop.create_task(publishtask())
aloop.create_task(immediateluxpublishtask())
aloop.create_task(mqtttask())
aloop.run_forever()

Sent 85 lines (2602 bytes) to main.py.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [2]:
%ls

Listing directory '/'.
      230    boot.py
      231    config.txt
     3353    main.py
      960    utils.py


In [ ]:
#%fetchfile --load main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine
from machine import Pin, PWM

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)

mApwm = PWM(Pin(int(fconfig["mAen"])))
mAfore = Pin(int(fconfig["mAfore"]), Pin.OUT)
mAback = Pin(int(fconfig["mAback"]), Pin.OUT)
mApwm.freq(10)
mApwm.duty(0)

mBpwm = PWM(Pin(int(fconfig["mBen"])))
mBfore = Pin(int(fconfig["mBfore"]), Pin.OUT)
mBback = Pin(int(fconfig["mBback"]), Pin.OUT)
mBpwm.freq(10)
mBpwm.duty(0)

timeZeroBy = 1
async def zeroingtask():
    global timeZeroBy
    while True:
        if timeZeroBy != 0:
            if time.ticks_ms() > timeZeroBy:
                timeZeroBy = 0
                mAfore.value(0)
                mAback.value(0)
                mBfore.value(0)
                mBback.value(0)
                pinled.value(1)
        await asyncio.sleep_ms(50)


def updatewheels(msg):
    global timeZeroBy
    try:
        s = msg.split()
        timeZeroBy = time.ticks_ms() + int(s[6])
        mApwm.duty(int(s[0]))
        mAfore.value(int(s[1]))
        mAback.value(int(s[2]))
        mBpwm.duty(int(s[3]))
        mBfore.value(int(s[4]))
        mBback.value(int(s[5]))
        pinled.value(0)
    except Exception as e:
        print("Bad wheels", msg, e)


boardname = fconfig["boardname"]
tboardcmd = boardname.encode() + b"/cmd"
tboardwheels = boardname.encode() + b"/wheels"

for l in ["500 1 0 500 1 0 1", "500 0 1 500 0 1 1", 
          "500 1 0 500 0 1 1", "500 0 1 500 1 0 1", "500 0 0 500 0 0 1"]:
    updatewheels(l)
    time.sleep_ms(500)

async def callbackcmdtask(cmd):
    print("executing", [cmd])
    pinled.value(0)
    try:
        exec(cmd)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)

def callbackcmd(topic, msg, retained):
    if topic == tboardcmd:
        aloop.create_task(callbackcmdtask(msg))
    else:
        updatewheels(msg)
        
async def onconnecttask(client):
    print("subscribing")
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True, qos=1)
    print("subscribing to", tboardcmd, tboardwheels)
    await client.subscribe(tboardcmd, 1)
    await client.subscribe(tboardwheels, 1)
    
async def mqtttask(bflip=False):
    print("mqttttask", bflip)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        await client.connect()
        return
    except OSError as e:
        print("client connect fail", [e])
    aloop.create_task(mqtttask(True))
        
        
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(mqtttask())
aloop.create_task(zeroingtask())
aloop.run_forever()
